In [1]:
import os
os.chdir('/Users/bnowacki/Documents/Git Repositories/rapid-soh-estimation-from-short-pulses')

from rapid_soh_estimation.rapid_soh_estimation.config import *
from rapid_soh_estimation.rapid_soh_estimation.common_methods import *


In [2]:
cc_data = load_processed_data(data_type='cc')
pulse_data = load_processed_data(data_type='slowpulse')

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, root_mean_squared_error
from sklearn.preprocessing import StandardScaler
import keras
from copy import copy, deepcopy


def get_cell_ids_in_group(group_id:int) -> np.ndarray:
	"""Gets all cell ids in the specified group id

	Args:
		group_id (int): The id of the group for which to return the cell ids 

	Returns:
		np.ndarray: An array of all cell_ids in the specified group
	"""
	assert group_id in df_test_tracker['Group'].unique(), f"Invalid group id entered: {group_id}. The group id must be one of the following: {df_test_tracker['Group'].unique()}"
	start = (group_id - 1) * 6
	end = min(start+6, 64)
	return np.arange(start, end, 1) + 2

class Custom_CVSplitter():
	"""A custom cross-validation split wrapper. Allows for splitting by group_id or cell_id and returns n_splits number of cross validation folds
	"""

	def __init__(self, n_splits=3, split_type='group_id', rand_seed=None):
		assert isinstance(n_splits, int), "\'n_splits\' must be an interger value"
		self.n_splits = n_splits
		self.allowed_split_types = ['group_id', 'cell_id']
		assert split_type in self.allowed_split_types, "ValueError. \'split type\' must be one of the following: {}".format(self.allowed_split_types)
		self.split_type = split_type
		self.rand_seed = rand_seed
		
	def get_n_splits(self, X, y, groups):
		return self.n_splits

	def split(self, X, y, cell_ids):
		'given input data (X) and output data (y), returns (train_idxs, test_idxs) --> idxs are relative to X & y'
		kf = None
		if self.rand_seed is None:
			kf = KFold(n_splits=self.n_splits, shuffle=True)
		else:
			kf = KFold(n_splits=self.n_splits, shuffle=True, random_state=self.rand_seed)
		
		if self.split_type == self.allowed_split_types[0]:      # 'group_id'
			group_ids = np.arange(1, 12, 1)
			# for every cv split (by group), convert group_id_idxs to X & y idxs
			for train_group_idxs, test_group_idxs in kf.split(group_ids):
				train_idxs = []
				test_idxs = []
				train_groups = group_ids[train_group_idxs]
				test_groups = group_ids[test_group_idxs]
				# go through all train group ids in this split
				for train_group_id in train_groups: 
					train_cell_ids = get_cell_ids_in_group(train_group_id)
					# add X & y idxs where cell_id is equal to each cell in this group
					for cell_id in train_cell_ids:
						cell_idxs = np.hstack(np.argwhere( cell_ids == cell_id ))
						train_idxs.append(cell_idxs)
				# go through all test group ids in this split
				for test_group_id in test_groups:
					test_cell_ids = get_cell_ids_in_group(test_group_id)
					# add X & y idxs where cell_id is equal to each cell in this group
					for cell_id in test_cell_ids:
						cell_idxs = np.hstack(np.argwhere( cell_ids == cell_id ))
						test_idxs.append(cell_idxs)

				train_idxs = np.hstack(train_idxs)
				test_idxs = np.hstack(test_idxs)
				yield train_idxs, test_idxs
				
		elif self.split_type == self.allowed_split_types[1]:      # 'cell_id'
			# for every cv split (by cell), convert cell_id_idxs to X & y idxs
			for train_cell_idxs, test_cell_idxs in kf.split(np.unique(cell_ids)):
				train_idxs = []
				test_idxs = []
				train_cells = np.unique(cell_ids)[train_cell_idxs]
				test_cells = np.unique(cell_ids)[test_cell_idxs]
				
				# go through all train group ids in this split
				for train_cell_id in train_cells: 
					cell_idxs = np.hstack(np.argwhere(cell_ids == train_cell_id))
					train_idxs.append(cell_idxs)
					
				# go through all test group ids in this split
				for test_cell_id in test_cells:
					cell_idxs = np.hstack(np.argwhere(cell_ids == test_cell_id))
					test_idxs.append(cell_idxs)
			
				train_idxs = np.hstack(train_idxs)
				test_idxs = np.hstack(test_idxs)
				yield train_idxs, test_idxs



def create_modeling_data(all_data:dict, input_feature_keys:list, output_feature_keys:list=['q_dchg', 'dcir_chg_20', 'dcir_chg_50', 'dcir_chg_90', 'dcir_dchg_20', 'dcir_dchg_50', 'dcir_dchg_90']) -> dict:
	"""Returns new dictionary with 'model_input' and 'model_output' keys for simpler model training

	Args:
		all_data (dict): All data from which to extrac the specified input and output feature
		input_feature_keys (list): A list of keys (that exist in all_data.keys()) to use as model input
		output_feature_keys (list, optional): A list of keys (that exist in all_data.keys()) to use as model output. Defaults to ['q_dchg', 'dcir_chg_20', 'dcir_chg_50', 'dcir_chg_90', 'dcir_dchg_20', 'dcir_dchg_50', 'dcir_dchg_90'].

	Returns:
		dict: A new dict with keys: ['cell_id', 'group_id', 'rpt', 'model_input', 'model_output']
	"""
	assert len(input_feature_keys) > 0
	for f in input_feature_keys: assert f in list(all_data.keys())
	for f in output_feature_keys: assert f in list(all_data.keys())

	modeling_dic = {
		'cell_id':all_data['cell_id'],
		'group_id':all_data['group_id'],
	 	'rpt':all_data['rpt'],
		'model_input':[],
		'model_output':[],
	}
	if len(input_feature_keys) == 1:
		modeling_dic['model_input'] = all_data[input_feature_keys[0]]

	for i in range(len(all_data['cell_id'])):
		if len(input_feature_keys) > 1:
			modeling_dic['model_input'].append( [all_data[f_key][i] for f_key in input_feature_keys] )
		modeling_dic['model_output'].append( [all_data[f_key][i] for f_key in output_feature_keys] )

	modeling_dic['model_input'] = np.asarray(modeling_dic['model_input'])
	modeling_dic['model_output'] = np.asarray(modeling_dic['model_output'])
	return modeling_dic

def create_model(n_hlayers:int, n_neurons:int, act_fnc:str, opt_fnc:str, learning_rate:float, input_shape=(100,)) -> keras.models.Sequential:
	"""Builds a Keras neural network model (MLP) using the specified parameters. The model is optimized for accuracy. Make sure model outputs (if multiple target) are normalized, otherwise optimization will be biased towards one target variable.

	Args:
		n_hlayers (int): Number of fully-connected hidden layers
		n_neurons (int): Number of neurons per hidden layer
		act_fnc (str): Activation function to use (\'tanh\', \'relu\', etc)
		opt_fnc (str): {\'sgd\', \'adam\'} Optimizer function to use 
		learning_rate (float): Learning rate
		input_shape (int, optional): Input shape of model. Defaults to (100,).

	Raises:
		ValueError: _description_

	Returns:
		keras.models.Sequential: compiled Keras model
	"""

	# add input layer to Sequential model
	model = keras.models.Sequential()
	model.add( keras.Input(shape=input_shape) )

	# add hidden layers
	for i in range(n_hlayers):
		model.add( keras.layers.Dense(units=n_neurons, activation=act_fnc) )
		
	# add output layer
	model.add( keras.layers.Dense(7) )

	# compile model with chosen metrics
	opt = None
	if opt_fnc == 'adam':
		opt = keras.optimizers.Adam(learning_rate=learning_rate)
	elif opt_fnc == 'sgd':
		opt = keras.optimizers.SGD(learning_rate=learning_rate)
	else:
		raise ValueError("opt_func must be either \'adam\' or \'sgd\'")

	model.compile(optimizer=opt,
					loss=keras.losses.mean_squared_error,      
					# make sure to normalize all outputs, otherwise DCIR values will drastically skew MSE reading compared to error of predicted SOH
					metrics=['accuracy'] )
	return model

def get_prediction_error(y_true, y_predicted):
    '''returns tuple of (MAPE, RMSE)'''
    mape = []
    rmse = []
    # print("y_pred size: ", np.size(y_predicted, axis=1) )
    # print("y_true size: ", np.size(y_true, axis=1))
    if len(np.shape(y_true)) > 1:
        for i in range(0, np.size(y_predicted, axis=1)):
            mape.append(np.round(mean_absolute_percentage_error(y_true[:,i], y_predicted[:,i]), 4))
            rmse.append(np.round(root_mean_squared_error(y_true[:,i], y_predicted[:,i]), 4))
    else:
        mape.append(np.round(mean_absolute_percentage_error(y_true, y_predicted), 4))
        rmse.append(np.round(root_mean_squared_error(y_true, y_predicted), 4))
    mape = np.vstack(mape)
    rmse = np.vstack(rmse)
    return mape.reshape(-1), rmse.reshape(-1)



# build model
model = create_model(5, 100, 'relu', 'sgd', 0.010, input_shape=(100,))

# get modeling data
all_data = deepcopy(pulse_data)
idxs = np.where((all_data['pulse_type'] == 'chg') & (all_data['soc'] == 20))
for k in all_data.keys():
	all_data[k] = all_data[k][idxs]
all_data['voltage_rel'] = np.asarray([v - v[0] for v in all_data['voltage']])

modeling_data = create_modeling_data(all_data=all_data, input_feature_keys=['voltage_rel'])

# normalize modeling data inputs and outputs
modeling_data['input_scaler'] = StandardScaler()
modeling_data['model_input_scaled'] = modeling_data['input_scaler'].fit_transform(modeling_data['model_input'])
modeling_data['output_scaler'] = StandardScaler()
modeling_data['model_output_scaled'] = modeling_data['output_scaler'].fit_transform(modeling_data['model_output'])
    
# define early stop callback
early_stop = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=25, verbose=False, mode='auto', baseline=None, restore_best_weights=True)


# run multiple CVs and average estimation error
num_iters = 5
mape_avgs = []
for i in range(num_iters):
	# generate cross validation splits
	cvSplitter = Custom_CVSplitter(n_splits=3, split_type='group_id', rand_seed=i)
	cv_splits = cvSplitter.split(modeling_data['model_input'], modeling_data['model_output'], modeling_data['cell_id']) 
	cv_splits = list(cv_splits)
	mape_qdchg = []
	for cv_idx, (train_idxs, test_idxs) in enumerate(cv_splits):
		# get train and test data for this cv split
		X_train = modeling_data['model_input_scaled'][train_idxs]
		y_train = modeling_data['model_output_scaled'][train_idxs]
		X_test = modeling_data['model_input_scaled'][test_idxs]
		y_test = modeling_data['model_output_scaled'][test_idxs]

		# train model
		history = model.fit(
			X_train, y_train,
			validation_split = 0.1,
			batch_size = 50,
			epochs = 100,
			callbacks = early_stop, 
			verbose = False)

		y_pred = model.predict(X_test, verbose=False)
		errors = get_prediction_error(y_test, y_pred)
		
		mape_qdchg.append(errors[0][0])
	print(f"Iter {i}: Average MAPE of Q_dchg={round(np.average(np.asarray(mape_qdchg)), 4)}%")
	mape_avgs.append(np.average(np.asarray(mape_qdchg)))
print(f"Average of all runs: MAPE={round(np.average(np.asarray(mape_avgs)), 4)}")

In [ ]:
import optuna

# OPTUNA model optimization 

def optuna_create_model(trial):
	model = create_model(
		n_hlayers=		trial.suggest_int("n_layers", 1, 5),
		n_neurons=		trial.suggest_int("n_neurons", 8, 128),
		act_fnc=		trial.suggest_categorical("activation", ['relu', 'tanh']),
		opt_fnc=		trial.suggest_categorical("optimizer", ['sgd', 'adam']),
		learning_rate=	trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
		input_shape=	(100,)
	)
	return model

def optuna_objective(trial):
	train_idxs, test_idxs = cv_splits[0]
	X_train = modeling_data['model_input_scaled'][train_idxs]
	y_train = modeling_data['model_output_scaled'][train_idxs]
	X_test = modeling_data['model_input_scaled'][test_idxs]
	y_test = modeling_data['model_output_scaled'][test_idxs]
	
	model = optuna_create_model(trial)
	model.fit(
		X_train, y_train,
		validation_split = 0.1,
		batch_size = 50,
		epochs = 100,
		callbacks = early_stop, 
		verbose = False)
	
	y_pred = model.predict(X_test, verbose=False)
	err = get_prediction_error(y_test, y_pred)[0][0]	# MAPE of q_dchg prediction
	return err


N_TRIALS = 100
study = optuna.create_study(direction="minimize", study_name="pulse_model_optimization")
study.optimize(optuna_objective, n_trials=N_TRIALS, n_jobs=-1)
print('*'*100)
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("Best Params:")
print(study.best_params)
print('*'*100)

# get the optimized model
opt_model = optuna_create_model(study.best_trial)
opt_model.summary()